In [1]:
import torch
import os
from pathlib import Path
from torchvision import datasets, transforms
import matplotlib.pyplot as plt 
import cv2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tqdm import tqdm

In [2]:

transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])
dataset = datasets.ImageFolder(Path('/home/miza/plant-disease/src/analysis/data_analysis.ipynb')\
    .parent.parent.parent.joinpath('data').joinpath('leaves'),transform=transform)
    
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=15)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=15)   
def save_dataset():
    train_folder = Path('/home/miza/plant-disease/src/analysis/data_analysis.ipynb')\
        .parent.parent.parent.joinpath('data').joinpath('train')
    test_folder = Path('/home/miza/plant-disease/src/analysis/data_analysis.ipynb')\
        .parent.parent.parent.joinpath('data').joinpath('test')
    train_folder.mkdir(parents=True, exist_ok=True)
    test_folder.mkdir(parents=True, exist_ok=True)
    def save_tensors(loader, folder):
        for batch_idx, (images, labels) in tqdm(enumerate(loader),total=len(loader),desc=f'Saving {folder.name}'):
            for i in range(images.size(0)):
                img_tensor = images[i].cpu()
                label_tensor = torch.tensor(labels[i].item(), device='cpu')
                label_folder = folder / str(label_tensor.item())
                label_folder.mkdir(parents=True, exist_ok=True)
                torch.save(img_tensor, label_folder / f'{batch_idx * loader.batch_size + i}_image.pt')
                torch.save(label_tensor, label_folder / f'{batch_idx * loader.batch_size + i}_label.pt')
    save_tensors(train_loader, train_folder)
    save_tensors(test_loader, test_folder)

In [3]:
def load_dataset_to_memory(dataset, device):
    images = []
    labels = []
    for img, label in tqdm(dataset,total=len(dataset)):
        images.append(img.to(device))
        labels.append(torch.tensor(label, device=device))
    return torch.stack(images), torch.tensor(labels)

# train_images, train_labels = load_dataset_to_memory(train_dataset, device)
# test_images, test_labels = load_dataset_to_memory(test_dataset, device)

In [13]:

import torch.nn.functional as F
class Convolutional(torch.nn.Module):
    def __init__(self):
        super(Convolutional, self).__init__()
        self.conv1 = torch.nn.Conv2d(3,32,3,2,padding_mode='replicate')
        self.pool1= torch.nn.MaxPool2d(2)
        self.conv2 = torch.nn.Conv2d(32,64,3,2,padding_mode='replicate')
        self.pool2= torch.nn.MaxPool2d(2)
        self.conv3 = torch.nn.Conv2d(64,128,3,2,padding_mode='replicate')
        self.pool3= torch.nn.MaxPool2d(2)
        self.conv4 = torch.nn.Conv2d(128,256,3,2,padding_mode='replicate')     
        self.pool4= torch.nn.MaxPool2d(2)
        self.conv5 = torch.nn.Conv2d(256,512,3,2,padding_mode='replicate')
        
        self.fc1 = torch.nn.Linear(1152, 32)
        self.fc2 = torch.nn.Linear(32, 4)
    
    def forward(self,x):
        x= self.pool1(F.relu(self.conv1(x)))
        x= self.pool2(F.relu(self.conv2(x)))
        x= self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] 
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [14]:
model = Convolutional()

print(model)

Convolutional(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding_mode=replicate)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding_mode=replicate)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding_mode=replicate)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding_mode=replicate)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding_mode=replicate)
  (fc1): Linear(in_features=1152, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=4, bias=True)
)


In [16]:
model.to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
EPOCHS=10
running_loss=0.0
count = 1
for epoch in tqdm(range(EPOCHS),total = EPOCHS):
    print(f'Running loss for epoch {epoch + 1}: {running_loss/(count)}')
    count = 0
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device) 
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        count+=1


  0%|          | 0/10 [00:00<?, ?it/s]

Running loss for epoch 1: 0.0


 10%|█         | 1/10 [00:31<04:44, 31.65s/it]

Running loss for epoch 2: 1.344514504859322


 20%|██        | 2/10 [01:04<04:21, 32.64s/it]

...

 80%|████████  | 8/10 [05:17<01:21, 40.69s/it]

Running loss for epoch 9: 1.0729346698836277


 90%|█████████ | 9/10 [06:10<00:44, 44.41s/it]

Running loss for epoch 10: 1.0700499015419107


100%|██████████| 10/10 [06:56<00:00, 41.60s/it]


In [17]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # Przenieś dane na GPU
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test images: {100 * correct / total} %')

Accuracy on the test images: 67.60797342192691 %
